In [2]:
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
from dask.utils import SerializableLock
from dask.distributed import Client
client = Client('129.236.21.48:8786')
client

<Client: scheduler='tcp://129.236.21.48:8786' processes=1 cores=1>

In [3]:
ddir = '/swot/SUM01/LLC/llc_4320_agulhas/'
%time ds = xr.open_mfdataset(ddir + 'llc_4320_agulhas.000001*.nc')
sl = slice(0,256)
region_slice = dict(i=sl, j=sl, i_g=sl, j_g=sl)
reg = ds.isel(**region_slice)

CPU times: user 3.75 s, sys: 206 ms, total: 3.95 s
Wall time: 6.3 s


In [4]:
salt_sq_profile = (reg.Salt**2).mean(dim=('time', 'j', 'i'))
salt_sq_profile

/home/rpa/.conda/envs/dask_distributed/lib/python3.5/site-packages/dask/array/core.py:465: RuntimeWarning: overflow encountered in square
  o = func(*args, **kwargs)


<xarray.DataArray 'Salt' (k: 90)>
dask.array<mean_ag..., shape=(90,), dtype=float32, chunksize=(90,)>
Coordinates:
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
    face     int64 1

In [5]:
# without distributed
print(reg.Salt.nbytes / 1e9)
%time salt_sq_profile.load()

1.58072832
CPU times: user 4.14 s, sys: 1min 21s, total: 1min 25s
Wall time: 5min 43s


<xarray.DataArray 'Salt' (k: 90)>
array([ 1145.026611,  1145.012939,  1145.00647 ,  1145.002197,  1144.999268,
        1144.997314,  1144.995972,  1144.994995,  1144.994751,  1144.994995,
        1144.996338,  1145.000244,  1145.009155,  1145.023926,  1145.044312,
        1145.067505,  1145.091675,  1145.118774,  1145.144897,  1145.170532,
        1145.199585,  1145.239746,  1145.328003,  1145.771118,  1148.019531,
        1154.254517,  1163.293701,  1171.090454,  1175.198853,  1176.563843,
        1177.584839,  1179.678345,  1182.645874,  1185.531494,  1187.729004,
        1189.335815,  1190.763916,  1192.262207,  1193.797974,  1195.224365,
        1196.46106 ,  1197.53125 ,  1198.494385,  1199.386475,  1200.208862,
        1200.951416,  1201.612793,  1202.202393,  1202.732544,  1203.210205,
        1203.636475,  1204.009399,  1204.327515,  1204.590942,  1204.800781,
        1204.958008,  1205.06311 ,  1205.116455,  1205.120117,  1205.077271,
        1204.993408,  1204.874634,  1204.7

In [6]:
print(reg.Salt.nbytes / 1e9)
print(client)
%time salt_sq_profile.load()

1.58072832
<Client: scheduler='tcp://129.236.21.48:8786' processes=1 cores=1>
CPU times: user 315 ms, sys: 56.7 ms, total: 372 ms
Wall time: 8min 48s


<xarray.DataArray 'Salt' (k: 90)>
array([ 1145.026611,  1145.012939,  1145.00647 ,  1145.002197,  1144.999268,
        1144.997314,  1144.995972,  1144.994995,  1144.994751,  1144.994995,
        1144.996338,  1145.000244,  1145.009155,  1145.023926,  1145.044312,
        1145.067505,  1145.091675,  1145.118774,  1145.144897,  1145.170532,
        1145.199585,  1145.239746,  1145.328003,  1145.771118,  1148.019531,
        1154.254517,  1163.293701,  1171.090454,  1175.198853,  1176.563843,
        1177.584839,  1179.678345,  1182.645874,  1185.531494,  1187.729004,
        1189.335815,  1190.763916,  1192.262207,  1193.797974,  1195.224365,
        1196.46106 ,  1197.53125 ,  1198.494385,  1199.386475,  1200.208862,
        1200.951416,  1201.612793,  1202.202393,  1202.732544,  1203.210205,
        1203.636475,  1204.009399,  1204.327515,  1204.590942,  1204.800781,
        1204.958008,  1205.06311 ,  1205.116455,  1205.120117,  1205.077271,
        1204.993408,  1204.874634,  1204.7

In [28]:
client.shutdown()

In [7]:
# 8 minutes to read 1.5 GB of data
# one worker, one process, one thread
print('%g MB/s' % (1.5e3 / (8*60)))

3.125 MB/s


In [27]:
%time ds_single = xr.open_dataset(ddir + 'llc_4320_agulhas.0000010368.nc')

CPU times: user 42.7 ms, sys: 3.01 ms, total: 45.7 ms
Wall time: 44.6 ms


In [28]:
ds_single

<xarray.Dataset>
Dimensions:   (i: 2160, i_g: 2160, j: 2160, j_g: 2160, k: 90, k_l: 90, k_p1: 91, k_u: 90, time: 1)
Coordinates:
  * j         (j) float64 0.0 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 ...
  * k         (k) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j_g       (j_g) float64 0.0 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 ...
  * i         (i) int64 1080 1081 1082 1083 1084 1085 1086 1087 1088 1089 ...
  * k_p1      (k_p1) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k_u       (k_u) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * i_g       (i_g) int64 1080 1081 1082 1083 1084 1085 1086 1087 1088 1089 ...
  * k_l       (k_l) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
    face      int64 1
  * time      (time) timedelta64[ns] 3 days
    iter      (time) int64 10368
Data variables:
    PhiBot    (time, j, i) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    oceQnet   (time, j, i) float32 0.0 0

In [29]:
print(ds_single.Salt.nbytes / 1e9)
%time ds_single.Salt.load()

1.679616
CPU times: user 1.48 ms, sys: 647 ms, total: 649 ms
Wall time: 643 ms


<xarray.DataArray 'Salt' (time: 1, k: 90, j: 2160, i: 2160)>
array([[[[ 33.753368, ...,  33.915161],
         ..., 
         [ 36.108173, ...,   0.      ]],

        ..., 
        [[  0.      , ...,   0.      ],
         ..., 
         [  0.      , ...,   0.      ]]]], dtype=float32)
Coordinates:
  * j        (j) float64 0.0 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 ...
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * i        (i) int64 1080 1081 1082 1083 1084 1085 1086 1087 1088 1089 ...
    face     int64 1
  * time     (time) timedelta64[ns] 3 days
    iter     (time) int64 10368
Attributes:
    standard_name: SALT
    long_name: Salinity
    units: psu

In [30]:
print(ds_single.isel(**region_slice).Theta.nbytes/1e9)
%time ds_single.isel(**region_slice).Theta.load()

0.02359296
CPU times: user 16.7 ms, sys: 61 ms, total: 77.8 ms
Wall time: 76.9 ms


<xarray.DataArray 'Theta' (time: 1, k: 90, j: 256, i: 256)>
array([[[[-1.318675, ..., -1.13113 ],
         ..., 
         [ 0.64202 , ...,  0.445646]],

        ..., 
        [[ 0.      , ...,  0.      ],
         ..., 
         [ 0.      , ...,  0.      ]]]], dtype=float32)
Coordinates:
  * j        (j) float64 0.0 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 ...
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * i        (i) int64 1080 1081 1082 1083 1084 1085 1086 1087 1088 1089 ...
    face     int64 1
  * time     (time) timedelta64[ns] 3 days
    iter     (time) int64 10368
Attributes:
    standard_name: THETA
    long_name: Potential Temperature
    units: degC

In [21]:
region_slice

{'i': slice(0, 256, None),
 'i_g': slice(0, 256, None),
 'j': slice(0, 256, None),
 'j_g': slice(0, 256, None)}